In [1]:
import pandas as pd
import numpy as np

In [27]:
episode_basics_df = pd.read_csv(r"C:\Users\19692\Downloads\CSE560-Project\sampled_data\epiosde_basic_sampled.tsv", sep="\t")
episode_basics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881413 entries, 0 to 881412
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          881413 non-null  object
 1   primaryTitle    881413 non-null  object
 2   originalTitle   881413 non-null  object
 3   startYear       881413 non-null  object
 4   runtimeMinutes  881413 non-null  object
dtypes: object(5)
memory usage: 33.6+ MB


In [28]:
# replace "," or ";" to "/" in the string
episode_basics_df['primaryTitle'] = episode_basics_df['primaryTitle'].str.replace(r',','/', regex=False)
episode_basics_df['primaryTitle'] = episode_basics_df['primaryTitle'].str.replace(r';','/', regex=False)
episode_basics_df['originalTitle'] = episode_basics_df['originalTitle'].str.replace(r',','/', regex=False)
episode_basics_df['originalTitle'] = episode_basics_df['originalTitle'].str.replace(r';','/', regex=False)

In [29]:
# replace \N to ''
episode_basics_df['tconst'] = episode_basics_df['tconst'].replace(r'\N','')
episode_basics_df['primaryTitle'] = episode_basics_df['primaryTitle'].replace(r'\N','')
episode_basics_df['startYear'] = episode_basics_df['startYear'].replace(r'\N','')
episode_basics_df['runtimeMinutes'] = episode_basics_df['runtimeMinutes'].replace(r'\N','')

In [30]:
# delete data rows where "tconst" or "primaryTitle" is None based on the constraint.
episode_basics_df[["tconst", "primaryTitle"]].dropna()
episode_basics_df = episode_basics_df[episode_basics_df["primaryTitle"].isna()==False]

In [31]:
# some records have wrong information
# for example, put genres info in runtimeMinutes
# delete them
episode_basics_df = episode_basics_df[
    episode_basics_df['runtimeMinutes'].isna() |
    episode_basics_df['runtimeMinutes'].astype(str).str.isdigit() |
    (episode_basics_df['runtimeMinutes'] == '')
]



In [33]:
import psycopg2
import io

In [34]:
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres.rnxnnqsyjcircloxrryq",
    password="cse560courseproject",
    host="aws-0-us-east-1.pooler.supabase.com",
    port=6543
)
cur = conn.cursor()

## TV Movie Relation

In [41]:


# for tv_movie relation
buffer = io.StringIO()
episode_basics_df.to_csv(buffer, index=False, header=False)
buffer.seek(0)


0

In [40]:
conn.rollback()

In [42]:

# upload data to tv_movie
cur.copy_from(buffer, 'episode_details', sep=',', null='')
conn.commit()